<a href="https://colab.research.google.com/github/xsrv07/AI/blob/main/Twitter_Sentiment_Analysis_Part_2A_Other_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load the Tweets Dataset

In [ ]:
import nltk
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
import numpy as np
pd.set_option("display.max_colwidth", 200)

In [ ]:
tweets = pd.read_pickle("cleaned_tweets_v1.pkl")
tweets.head()

,label,tweet,cleaned_tweets,cleaned_tweets_without_stopwords
0,1,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone,fingerprint pregnancy test android apps beautiful cute health igers iphoneonly iphonesia iphone,fingerprint pregnancy test android apps beautiful cute health igers iphoneonly iphonesia iphone
1,1,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/,finally transparant silicon case thanks uncle yay sony xperia sonyexperias,finally transparant silicon case thanks uncle yay sony xperia sonyexperias
2,1,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu,love this would you talk makememories unplug relax iphone smartphone wifi connect,love talk makememories unplug relax iphone smartphone wifi connect
3,1,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/,wired know george wa made that way iphone cute daventry home,wired know george way iphone cute daventry home
4,0,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!,what amazing service apple will not even talk about question have unless pay them for their stupid support,amazing service apple talk question unless pay stupid support


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
X = tweets['cleaned_tweets']
y = tweets['label']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.25, random_state=42)

In [ ]:
y_train.value_counts(normalize=True)

label
1    0.744276
0    0.255724
Name: proportion, dtype: float64

In [ ]:
y_test.value_counts(normalize=True)

label
1    0.743939
0    0.256061
Name: proportion, dtype: float64

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
CV = CountVectorizer()
CV_train = CV.fit_transform(X_train)
CV_test = CV.transform(X_test)

In [ ]:
CV_train.shape  # 12898 unique words in the vocab created by using the X_train

(5940, 12937)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
TFIDF = TfidfVectorizer()
TFIDF_train = TFIDF.fit_transform(X_train)
TFIDF_test = TFIDF.transform(X_test)

In [ ]:
TFIDF_train.shape

(5940, 12937)

In [ ]:
# Best: CV, l2, C=0.05  >>
# 0.8874 0.0015
# 0.8769 0.0073
# no. of features = 1208

In [ ]:
import numpy as np

In [ ]:
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.svm import SVC

from sklearn.model_selection import cross_validate, StratifiedKFold

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
n_splits = 5
kfold = StratifiedKFold(n_splits=n_splits,shuffle=True, random_state=101)  # >> shuffler & spliiter

In [ ]:
TFIDF = TfidfVectorizer(max_df=0.5, min_df=10, max_features=None)
CVectorizer = CountVectorizer(max_df=0.5, min_df=10, max_features=None)
LR_l2 = LogisticRegression(solver='liblinear', class_weight='balanced', penalty='l2', C=0.05)

TFIDF_l2_pipeline = Pipeline([("TFIDF", TFIDF), ("LR", LR_l2)])
CV_l2_pipeline = Pipeline([("CV", CVectorizer), ("LR", LR_l2)])

LR_l1 = LogisticRegression(solver='liblinear', class_weight='balanced', penalty='l1', C=0.2)
TFIDF_l1_pipeline = Pipeline([("TFIDF", TFIDF), ("LR", LR_l1)])
CV_l1_pipeline = Pipeline([("CV", CVectorizer), ("LR", LR_l1)])

In [ ]:
# !pip install -Uq lightgbm

In [ ]:
### Create 6 models!!
models = []
models.append(("TFIDF_l2_pipeline", TFIDF_l2_pipeline))
models.append(("CV_l2_pipeline", CV_l2_pipeline))
models.append(("TFIDF_l1_pipeline", TFIDF_l1_pipeline))
models.append(("CV_l1_pipeline", CV_l1_pipeline))

KNN_pipeline = Pipeline([("CV", CVectorizer), ("KNN", KNeighborsClassifier())])
models.append(('KNN_pipleline', KNN_pipeline))

DT_model = DecisionTreeClassifier(max_depth=4, class_weight='balanced', random_state=100)
DT_pipeline = Pipeline([("CV", CVectorizer), ("DT_model", DT_model)])
models.append(('DT_pipeline', DT_pipeline))

models

[('TFIDF_l2_pipeline',
  Pipeline(steps=[('TFIDF', TfidfVectorizer(max_df=0.5, min_df=10)),
                  ('LR',
                   LogisticRegression(C=0.05, class_weight='balanced',
                                      solver='liblinear'))])),
 ('CV_l2_pipeline',
  Pipeline(steps=[('CV', CountVectorizer(max_df=0.5, min_df=10)),
                  ('LR',
                   LogisticRegression(C=0.05, class_weight='balanced',
                                      solver='liblinear'))])),
 ('TFIDF_l1_pipeline',
  Pipeline(steps=[('TFIDF', TfidfVectorizer(max_df=0.5, min_df=10)),
                  ('LR',
                   LogisticRegression(C=0.2, class_weight='balanced',
                                      penalty='l1', solver='liblinear'))])),
 ('CV_l1_pipeline',
  Pipeline(steps=[('CV', CountVectorizer(max_df=0.5, min_df=10)),
                  ('LR',
                   LogisticRegression(C=0.2, class_weight='balanced',
                                      penalty='l1', solver=

In [ ]:
# Add Naive Bayes (Multinomial, as it's suitable for text data with CountVectorizer features)
NB_model = MultinomialNB()
NB_pipeline = Pipeline([("CV", CVectorizer), ("NB_model", NB_model)])
models.append(('NB_pipeline', NB_pipeline))

# Add Random Forest
RF_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=100)
RF_pipeline = Pipeline([("CV", CVectorizer), ("RF_model", RF_model)])
models.append(('RF_pipeline', RF_pipeline))

# Add XGBoost
# from xgboost import XGBClassifier
# XGB_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=100)
# XGB_pipeline = Pipeline([("CV", CVectorizer), ("to_float32", convert_to_float32), ("XGB_model", XGB_model)])
# models.append(('XGB_pipeline', XGB_pipeline))

In [ ]:
# Add LightGBM
from lightgbm import LGBMClassifier
from sklearn.preprocessing import FunctionTransformer

# Add LightGBM with feature conversion to float
LGBM_model = LGBMClassifier(class_weight='balanced', random_state=100)

# FunctionTransformer ensures the input features are cast to float32
convert_to_float32 = FunctionTransformer(lambda x: x.astype(np.float32), validate=False)

LGBM_pipeline = Pipeline([
    ("CV", CVectorizer),
    ("to_float32", convert_to_float32),
    ("LGBM_model", LGBM_model)
])
models.append(('LGBM_pipeline', LGBM_pipeline))


models # we have created a list of models which will be tried!!

[('TFIDF_l2_pipeline',
  Pipeline(steps=[('TFIDF', TfidfVectorizer(max_df=0.5, min_df=10)),
                  ('LR',
                   LogisticRegression(C=0.05, class_weight='balanced',
                                      solver='liblinear'))])),
 ('CV_l2_pipeline',
  Pipeline(steps=[('CV', CountVectorizer(max_df=0.5, min_df=10)),
                  ('LR',
                   LogisticRegression(C=0.05, class_weight='balanced',
                                      solver='liblinear'))])),
 ('TFIDF_l1_pipeline',
  Pipeline(steps=[('TFIDF', TfidfVectorizer(max_df=0.5, min_df=10)),
                  ('LR',
                   LogisticRegression(C=0.2, class_weight='balanced',
                                      penalty='l1', solver='liblinear'))])),
 ('CV_l1_pipeline',
  Pipeline(steps=[('CV', CountVectorizer(max_df=0.5, min_df=10)),
                  ('LR',
                   LogisticRegression(C=0.2, class_weight='balanced',
                                      penalty='l1', solver=

In [ ]:
cv_results = []
n_splits = 5
mean_train_score = []
test_scores = []
names = []

# going to cross-validate all the 5 models
for name, model in models:
    results = cross_validate(model, X.values, y, return_train_score=True, cv=kfold, scoring="accuracy", error_score='raise')  # 'f1'
    cv_results.append(results)
    test_scores.append(results['test_score'] )
    mean_train_score.append(results['train_score'].mean().round(3) )
    names.append(name)
    print(name, ".... done!")

TFIDF_l2_pipeline .... done!
CV_l2_pipeline .... done!
TFIDF_l1_pipeline .... done!
CV_l1_pipeline .... done!
KNN_pipleline .... done!
DT_pipeline .... done!
NB_pipeline .... done!
RF_pipeline .... done!
[LightGBM] [Info] Number of positive: 4715, number of negative: 1621
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004834 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1627
[LightGBM] [Info] Number of data points in the train set: 6336, number of used features: 577
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Info] Number of positive: 4715, number of negative: 1621
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004981 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if me

In [ ]:
results_df = pd.DataFrame(test_scores, index=names, columns='CV1 CV2 CV3 CV4 CV5'.split() )
results_df['CV Mean'] = np.round(results_df.iloc[:,0:n_splits].mean(axis=1), 4)
results_df['CV Std Dev'] = np.round(results_df.iloc[:,0:n_splits].std(axis=1), 4)

results_df['Mean_Train_Score'] = mean_train_score

results_df.sort_values(by='CV Mean', ascending=False)*100

,CV1,CV2,CV3,CV4,CV5,CV Mean,CV Std Dev,Mean_Train_Score
LGBM_pipeline,87.689394,89.015152,88.952020,87.689394,88.005051,88.27,0.66,92.1
NB_pipeline,87.626263,89.141414,87.752525,87.815657,87.815657,88.03,0.63,88.7
CV_l2_pipeline,87.373737,88.762626,87.310606,87.563131,86.994949,87.60,0.68,88.7
RF_pipeline,87.563131,87.563131,87.689394,87.941919,86.931818,87.54,0.37,99.9
CV_l1_pipeline,87.058081,87.752525,87.121212,87.436869,86.994949,87.27,0.32,88.3
TFIDF_l2_pipeline,86.300505,87.373737,85.858586,86.047980,86.426768,86.40,0.59,87.0
TFIDF_l1_pipeline,84.280303,85.479798,83.964646,84.848485,84.406566,84.60,0.59,85.3
KNN_pipleline,79.734848,80.618687,79.797980,79.797980,79.229798,79.84,0.50,85.0
DT_pipeline,75.126263,71.275253,69.381313,76.893939,71.717172,72.88,3.05,72.9


In [ ]:
cv_results

[{'fit_time': array([0.05745816, 0.05382442, 0.0543716 , 0.05272365, 0.05288243]),
  'score_time': array([0.01207924, 0.01113057, 0.0114553 , 0.01139569, 0.01148152]),
  'test_score': array([0.86300505, 0.87373737, 0.85858586, 0.8604798 , 0.86426768]),
  'train_score': array([0.87042298, 0.86821338, 0.86963384, 0.87373737, 0.86916035])},
 {'fit_time': array([0.05954313, 0.05902052, 0.05752397, 0.05687809, 0.06109428]),
  'score_time': array([0.01093197, 0.01087093, 0.01113486, 0.01103425, 0.01142311]),
  'test_score': array([0.87373737, 0.88762626, 0.87310606, 0.87563131, 0.86994949]),
  'train_score': array([0.88794192, 0.88699495, 0.8844697 , 0.88731061, 0.88825758])},
 {'fit_time': array([0.05714965, 0.058043  , 0.0574224 , 0.05701256, 0.05655146]),
  'score_time': array([0.01139784, 0.01136136, 0.01144814, 0.01135182, 0.01163864]),
  'test_score': array([0.84280303, 0.85479798, 0.83964646, 0.84848485, 0.84406566]),
  'train_score': array([0.85542929, 0.84911616, 0.85179924, 0.85558

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
from sklearn.preprocessing import FunctionTransformer
import numpy as np

# Define parameter grid
param_grid = {
    'LGBM_model__n_estimators': [100, 200, 300],
    'LGBM_model__max_depth': [3, 5, 7],
}

# Create pipeline
pipeline = Pipeline([
    ("CV", CVectorizer),
    ("to_float32", FunctionTransformer(lambda x: x.astype(np.float32))),
    ("LGBM_model", LGBMClassifier(class_weight='balanced', random_state=100))
])

grid_search = GridSearchCV(
   pipeline,
   param_grid,
   cv=kfold,
   scoring='accuracy',
   n_jobs=-1,
   verbose=2
)

# Fit and get best parameters
search.fit(X_train, y_train)
best_params = search.best_params_
best_score = search.best_score_

/home/jupyter-prashant/.local/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/jupyter-prashant/.local/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/jupyter-prashant/.local/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/jupyter-prashant/.local/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/jupyter-prashant/.local/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'e

[LightGBM] [Info] Number of positive: 3537, number of negative: 1215
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 3.587919 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1249
[LightGBM] [Info] Number of data points in the train set: 4752, number of used features: 446
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

/home/jupyter-prashant/.local/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/jupyter-prashant/.local/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/jupyter-prashant/.local/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/jupyter-prashant/.local/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/jupyter-prashant/.local/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'e


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 

/home/jupyter-prashant/.local/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/jupyter-prashant/.local/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/jupyter-prashant/.local/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/jupyter-prashant/.local/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 

In [ ]:
a